# Oriented RepPoints

* Official code uses MMDetection.
* MMDetection models are built from config files - example for [Oriented RepPoints](https://github.com/LiWentomng/OrientedRepPoints/blob/main/configs/dota/orientedreppoints_r50.py).
* From this config file, it can be notices the these modules are used:
    * [`OrientedRepPointsDetector`](https://github.com/LiWentomng/OrientedRepPoints/blob/main/mmdet/models/oriented_detectors/orientedreppoints_detector.py)
    * [`OrientedRepPointsHead`](https://github.com/LiWentomng/OrientedRepPoints/blob/main/mmdet/models/dense_heads/orientedreppoints_head.py)
    * [`OBBPointAssigner`](https://github.com/LiWentomng/OrientedRepPoints/blob/main/mmdet/core/bbox/assigners/oriented_point_assigner.py)
    * [`OBBMaxIoUAssigner`](https://github.com/LiWentomng/OrientedRepPoints/blob/main/mmdet/core/bbox/assigners/oriented_max_iou_assigner.py)
    * [`FocalLoss`](https://github.com/LiWentomng/OrientedRepPoints/blob/main/mmdet/models/losses/focal_loss.py)
    * [`OBBGIoULoss`](https://github.com/LiWentomng/OrientedRepPoints/blob/main/mmdet/models/losses/iou_loss.py)
    * [`SpatialBorderLoss`](https://github.com/LiWentomng/OrientedRepPoints/blob/main/mmdet/models/losses/spatial_border_loss.py)

## Check Project's Implementation

In [1]:
import torch
from einops import rearrange, repeat

from obb.model.oriented_reppoints import OrientedRepPointsHead, rep_point_to_img_space
from obb.utils.box_ops import min_area_rect

In [2]:
img_h, img_w = (256, 256)

features_per_map = 256
strides = {'P2': 4, 'P3': 8, 'P4': 16, 'P5': 32}
feature_maps = {name: torch.rand(1, features_per_map, img_h // stride, img_w // stride)
                for name, stride in strides.items()}

for name, feature_map in feature_maps.items():
    stride = strides[name]
    print(f'feature map {name} with stride {strides[name]:2} have shape: {list(feature_map.shape)}')


feature map P2 with stride  4 have shape: [1, 256, 64, 64]
feature map P3 with stride  8 have shape: [1, 256, 32, 32]
feature map P4 with stride 16 have shape: [1, 256, 16, 16]
feature map P5 with stride 32 have shape: [1, 256, 8, 8]


### Head Architecture

In [3]:
rotated_RepPoints_head = OrientedRepPointsHead()
multi_level_centers1 = []
multi_level_centers2 = []
for name, feature_map in feature_maps.items():
    stride = strides[name]
    h = feature_map.shape[2]
    w = feature_map.shape[3]

    rep_points1, rep_points2, classification = rotated_RepPoints_head(feature_map)
    
    # convert rep points to image space in order to calculate the losses
    rep_points1 = rep_point_to_img_space(rep_points1, stride)
    rep_points2 = rep_point_to_img_space(rep_points2, stride)

    # convert rep points out of the model to have shape accepted by conversion function "g"
    # rep_points1_for_conversion_function = rearrange(rep_points1, 'b (N yx) h w -> b h w N yx', b=1, N=9)
    # rep_points2_for_conversion_function = rearrange(rep_points2, 'b (N yx) h w -> b h w N yx', b=1, N=9)

    # apply conversion function to get oriented boxes
    flattened_ob1 = min_area_rect(rearrange(rep_points1, 'b (N yx) h w -> (b h w) N yx', b=1, N=9))
    flattened_ob2 = min_area_rect(rearrange(rep_points2, 'b (N yx) h w -> (b h w) N yx', b=1, N=9))
    ob1 = rearrange(flattened_ob1, '(b h w) n yx -> b h w n yx', b=1, h=h, w=w)
    ob2 = rearrange(flattened_ob2, '(b h w) n yx -> b h w n yx', b=1, h=h, w=w)

    # find rep points centers for assigner
    centers1 = rearrange(rep_points1[0, 8:10, :, :], 'yx w h -> (w h) yx')
    centers2 = rearrange(rep_points2[0, 8:10, :, :], 'yx w h -> (w h) yx')
    stride_vec = torch.ones(centers1.shape[0], 1) * stride
    multi_level_centers1.append(torch.cat([centers1, stride_vec], dim=1)) # [..., 3]
    multi_level_centers2.append(torch.cat([centers2, stride_vec], dim=1)) # [..., 3]
    
    print('\n\t'.join([
        f'\nfeature map {name} with {stride = }:',
        f'{feature_map.shape = }',
        f'{classification.shape = }',
        f'{rep_points1.shape = }',
        f'{rep_points2.shape = }',
        f'{ob1.shape = }',
        f'{ob2.shape = }',
    ]))


feature map P2 with stride = 4:
	feature_map.shape = torch.Size([1, 256, 64, 64])
	classification.shape = torch.Size([1, 15, 64, 64])
	rep_points1.shape = torch.Size([1, 18, 64, 64])
	rep_points2.shape = torch.Size([1, 18, 64, 64])
	ob1.shape = torch.Size([1, 64, 64, 4, 2])
	ob2.shape = torch.Size([1, 64, 64, 4, 2])

feature map P3 with stride = 8:
	feature_map.shape = torch.Size([1, 256, 32, 32])
	classification.shape = torch.Size([1, 15, 32, 32])
	rep_points1.shape = torch.Size([1, 18, 32, 32])
	rep_points2.shape = torch.Size([1, 18, 32, 32])
	ob1.shape = torch.Size([1, 32, 32, 4, 2])
	ob2.shape = torch.Size([1, 32, 32, 4, 2])

feature map P4 with stride = 16:
	feature_map.shape = torch.Size([1, 256, 16, 16])
	classification.shape = torch.Size([1, 15, 16, 16])
	rep_points1.shape = torch.Size([1, 18, 16, 16])
	rep_points2.shape = torch.Size([1, 18, 16, 16])
	ob1.shape = torch.Size([1, 16, 16, 4, 2])
	ob2.shape = torch.Size([1, 16, 16, 4, 2])

feature map P5 with stride = 32:
	feature

[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


In [4]:
class OBBPointAssigner:
    """
    Assign a corresponding oriented gt box or background to each point.

    Source:
        https://github.com/LiWentomng/OrientedRepPoints/blob/main/mmdet/core/bbox/assigners/oriented_point_assigner.py

    Each proposals will be assigned with `0`, or a positive integer
    indicating the ground truth index.
    - 0: negative sample, no assigned gt
    - positive integer: positive sample, index (1-based) of assigned gt
    """

    def __init__(self, scale=4, pos_num=3):
        self.scale = scale
        self.pos_num = pos_num

    def assign(self, points, gt_obboxes, gt_labels=None):

        """Assign oriented gt boxes to points.
        This method assign a gt obox to every points set, each points set
        will be assigned with  0, or a positive number.
        0 means negative sample, positive number is the index (1-based) of
        assigned gt.
        The assignment is done in following steps, the order matters.
        1. assign every points to 0
        2. A point is assigned to some gt bbox if
            (i) the point is within the k closest points to the gt bbox
            (ii) the distance between this point and the gt is smaller than
                other gt bboxes
        Args:
            points (Tensor): points to be assigned, shape(n, 3) while last
                dimension stands for (x, y, stride).
            gt_oboxes (Tensor): Groundtruth oriented boxes, shape (k, 8).
            gt_labels (Tensor, optional): Label of gt_bboxes, shape (k, ).
        Returns:
            :obj:`AssignResult`: The assign results.
        """

        num_points = points.shape[0]
        num_gts = gt_obboxes.shape[0]

        if num_gts == 0 or num_points == 0:
            # If no truth assign everything to the background
            assigned_gt_inds = points.new_full((num_points,), 0, dtype=torch.long)
            if gt_labels is None:
                assigned_labels = None
            else:
                assigned_labels = points.new_zeros((num_points,), dtype=torch.long)
            return assigned_gt_inds, assigned_labels

        points_xy = points[:, :2]
        points_stride = points[:, 2]
        points_lvl = torch.log2(points_stride).int()  # [3...,4...,5...,6...,7...]
        lvl_min, lvl_max = points_lvl.min(), points_lvl.max()

        assert gt_obboxes.size(1) == 8, 'gt_obboxes should be (N * 8)'

        # gt_obboxes convert to gt_bboxes
        gt_xs, gt_ys = gt_obboxes[:, 0::2], gt_obboxes[:, 1::2]
        gt_xmin, _ = gt_xs.min(1)
        gt_ymin, _ = gt_ys.min(1)
        gt_xmax, _ = gt_xs.max(1)
        gt_ymax, _ = gt_ys.max(1)
        gt_bboxes = torch.cat([gt_xmin[:, None], gt_ymin[:, None],
                               gt_xmax[:, None], gt_ymax[:, None]], dim=1)

        # assign gt rbox
        gt_bboxes_xy = (gt_bboxes[:, :2] + gt_bboxes[:, 2:]) / 2

        gt_bboxes_wh = (gt_bboxes[:, 2:] - gt_bboxes[:, :2]).clamp(min=1e-6)
        scale = self.scale
        gt_bboxes_lvl = ((torch.log2(gt_bboxes_wh[:, 0] / scale) +
                          torch.log2(gt_bboxes_wh[:, 1] / scale)) / 2).int()
        gt_bboxes_lvl = torch.clamp(gt_bboxes_lvl, min=lvl_min, max=lvl_max)

        # stores the assigned gt index of each point
        assigned_gt_inds = points.new_zeros((num_points,), dtype=torch.long)

        # stores the assigned gt dist (to this point) of each point
        assigned_gt_dist = points.new_full((num_points,), float('inf'))
        points_range = torch.arange(points.shape[0])

        for idx in range(num_gts):
            gt_lvl = gt_bboxes_lvl[idx]

            # get the index of points in this level
            lvl_idx = gt_lvl == points_lvl
            points_index = points_range[lvl_idx]

            # get the points in this level
            lvl_points = points_xy[lvl_idx, :]

            # get the center point of gt
            gt_point = gt_bboxes_xy[[idx], :]

            # get width and height of gt
            gt_wh = gt_bboxes_wh[[idx], :]

            # compute the distance between gt center and all points in this level
            points_gt_dist = ((lvl_points - gt_point) / gt_wh).norm(dim=1)

            # find the nearest k points to gt center in this level
            min_dist, min_dist_index = torch.topk(points_gt_dist, self.pos_num, largest=False)

            # the index of nearest k points to gt center in this level
            min_dist_points_index = points_index[min_dist_index]

            # The less_than_recorded_index stores the index
            #   of min_dist that is less then the assigned_gt_dist. Where
            #   assigned_gt_dist stores the dist from previous assigned gt
            #   (if exist) to each point.
            less_than_recorded_index = min_dist < assigned_gt_dist[min_dist_points_index]

            # The min_dist_points_index stores the index of points satisfy:
            #   (1) it is k nearest to current gt center in this level.
            #   (2) it is closer to current gt center than other gt center.
            min_dist_points_index = min_dist_points_index[less_than_recorded_index]

            # assign the result
            assigned_gt_inds[min_dist_points_index] = idx + 1
            assigned_gt_dist[min_dist_points_index] = min_dist[less_than_recorded_index]

        if gt_labels is not None:
            assigned_labels = assigned_gt_inds.new_zeros((num_points,))
            pos_inds = torch.nonzero(assigned_gt_inds > 0).squeeze()
            if pos_inds.numel() > 0:
                assigned_labels[pos_inds] = gt_labels[
                    assigned_gt_inds[pos_inds] - 1]
        else:
            assigned_labels = None

        return assigned_gt_inds, assigned_labels


points1 = torch.cat(multi_level_centers1, dim=0)

# fake ground truth data
gt_obboxes = torch.tensor([[1, 1, 1, 10, 10, 10, 10, 1]])
gt_labels = torch.tensor([1])

assigner = OBBPointAssigner()
assigner.assign(points1, gt_obboxes, gt_labels)

(tensor([0, 0, 0,  ..., 0, 0, 0]), tensor([0, 0, 0,  ..., 0, 0, 0]))